<a href="https://colab.research.google.com/github/SauravUpadhyaya/WQU-MachineLearning/blob/main/DocumentSummarization_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
### importing the necessary libraries

from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

In [109]:
### Loading, for instance, text file now (note: can be in any format, changes can be made as per requirement. Also, by checking the file name, we can run the algorithm, if we don't want to check once at a time)  
### engine='python' is required to use because the engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex)
### If error_bad_lines=False is assigned to be True, then lines with too many fields (e.g. a csv line with too many commas) will by default cause an exception to be raised, and no DataFrame gets returned. So, to drop these bad lines and to return the DataFrame, it should be assigned False. 

import io
import os
from google.colab import files
uploaded = files.upload()
data = pd.read_csv(io.BytesIO(uploaded['myInterest.txt']), encoding='utf-8', engine='python',  error_bad_lines=False)


Saving myInterest.txt to myInterest (18).txt


Skipping line 2: unexpected end of data


In [101]:
data.head(50)

,The first paper in the field was published in 1958 (Lun),and it used a statistical technique. By 2016,term frequency-inverse document frequency was being employed. By 2016,pattern-based summarizing had emerged as the most powerful alternative for multi-document summarization. Latent semantic analysis (LSA) paired with non-negative matrix factorization overtook it the next year (NMF). Machine learning algorithms dominated the extractive summary of single texts by 2019,despite the fact that they did not replace other approaches and were frequently integrated with them. By 2020,the area was still thriving,with research focusing on abstractive summarizing and real-time summarization.


In [102]:
import nltk
nltk.download('punkt')
s = ""
d = {}
for a in data:
      s += a
# print s
sentences = sent_tokenize(s)
clean_sentences = []
for s in sentences:
    temp = re.sub("[^a-zA-Z]"," ",s)
    temp = temp.lower()
    clean_sentences.append(temp)
    d[temp] = s 
print(clean_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['the first paper in the field was published in       lun  and it used a statistical technique ', 'by      term frequency inverse document frequency was being employed ', 'by      pattern based summarizing had emerged as the most powerful alternative for multi document summarization ', 'latent semantic analysis  lsa  paired with non negative matrix factorization overtook it the next year  nmf  ', 'machine learning algorithms dominated the extractive summary of single texts by      despite the fact that they did not replace other approaches and were frequently integrated with them ', 'by      the area was still thriving with research focusing on abstractive summarizing and real time summarization ']


In [103]:
### A functiom is defined in order to remove the stop words that are downloaded from NLTk's list of english stop words.

import nltk
nltk.download('stopwords')
stop_words = stopwords.words('english')
def rem_stop(s):
    var = ""
    words = nltk.word_tokenize(s)
    for w in words:
        if( w not in stop_words):
           var+=w+" "
    return var

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [104]:
### removing the stop words using the function defined above

dict = {}
clean = []
# print clean_sentences
for s in clean_sentences:
    temp = rem_stop(s)
    clean.append(temp)
    dict[temp] = d[s]
print(clean)

['first paper field published lun used statistical technique ', 'term frequency inverse document frequency employed ', 'pattern based summarizing emerged powerful alternative multi document summarization ', 'latent semantic analysis lsa paired non negative matrix factorization overtook next year nmf ', 'machine learning algorithms dominated extractive summary single texts despite fact replace approaches frequently integrated ', 'area still thriving research focusing abstractive summarizing real time summarization ']


In [105]:
### loading and unzipping 


import tensorflow
from gensim import models
word2vec_path = 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [106]:
### creating vector representation of sentences after extracting word vectors

print(word2vec)
word_embeddings = {}
words = list(word2vec.wv.vocab)
print(len(words))
for a in words:
    word_embeddings[a]=word2vec[a]

print(len(word_embeddings))


sentence_vectors = []
for i in clean:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((300,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((300,))
  sentence_vectors.append(v)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


3000000
3000000


In [107]:
### generating the final summary after producing the graph using networkx and applying pagerank algorithm
### Graphs are formed where sentences are the nodes and edges are formed using Cosine Similarity between the sentences

sentence_similarity_martix = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sentence_similarity_martix[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,300), sentence_vectors[j].reshape(1,300))[0,0]

sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
scores = nx.pagerank(sentence_similarity_graph)
ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(clean)), reverse=True)

In [111]:
### printing the top ranked sentences

for i in range(2):
     print(dict[ranked_sentence[i][1]])

Machine learning algorithms dominated the extractive summary of single texts by 2019 despite the fact that they did not replace other approaches and were frequently integrated with them.
By 2016 pattern-based summarizing had emerged as the most powerful alternative for multi-document summarization.
